In [17]:
import kagglehub
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

# Regression Classifier.
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import pandas as pd
import pickle

# Statsmodels
import statsmodels.api as sm
import pandas as pd

In [18]:
# Load dataset.
# Download latest version
path = kagglehub.dataset_download("lashagoch/life-expectancy-who-updated")
print("Path to dataset files:", path)

file = "Life-Expectancy-Data-Updated.csv"

# TODO:  Correct features and assign target.
data = pd.read_csv(path + "/" + file)
data.columns
X = data[['Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed']]

y = data['Life_expectancy']

Path to dataset files: /Users/nathanlonghurst/.cache/kagglehub/datasets/lashagoch/life-expectancy-who-updated/versions/1


In [ ]:
# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Logistic Regression classifier instance.
model = RandomForestRegressor(oob_score=True, random_state=42)

# number of features
d = len(X_train.iloc[0])

# Specify values for certain hyperparameters
param_grid = {"max_leaf_nodes": [30, 40, 50], # max number of leaf nodes
              "n_estimators": [50, 100, 150, 200], # number of trees
              "max_depth": [5, 10, 15], # maximum depth on any branch in tree
              "min_samples_leaf": [20, 50, 100], # minimum number of samples in a leaf
              "max_features": [1, 3, int(np.floor(np.log2(d) + 1)), 'log2', 'sqrt'], # max number of features to consider in a tree
              }

# Perform a grid search to find the best parameters
model_gs = GridSearchCV(model, param_grid, scoring="neg_mean_squared_error", n_jobs=-1)

model_gs.fit(X_train, y_train)


print(f"best_parameters: {model_gs.best_params_}")
print(f"best score: {model_gs.best_score_}")
print(f"best oob score: {model_gs.best_estimator_.oob_score_}")
print(f"most important features: {model_gs.best_estimator_.feature_importances_}")

model_gs.best_estimator_.score(X_test,y_test)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
# Save the model.
with open('RandomForestRegressor.pkl', 'wb') as f:
    pickle.dump(model_gs.best_estimator_, f)

In [15]:
with open('RandomForestRegressor.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [16]:
y_test_hat = loaded_model.predict(X_test)
error = mean_squared_error(y_test, y_test_hat)
print(f"mean squared error: {error}")

mean squared error: 0.25806925654450436
